# Part2.  Emotion Recognition on Twitter 

## 1. read data

In [92]:
# libraries import
import os
import re
import time
import math
import json
import string
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#from gensim.models import KeyedVectors
#import helpers.data_mining_helpers as dmh

2022-11-18 08:44:11.389585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 08:44:13.271638: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-18 08:44:18.490782: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-18 08:44:18.491276: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [74]:
# nltk library
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [2]:
dir_data = 'Twitter_kaggle_data'

f_data_identification = os.path.join(dir_data, 'data_identification.csv')
data_identification = pd.read_csv(f_data_identification)
data_identification.head()

,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train


In [3]:
f_emotion = os.path.join(dir_data, 'emotion.csv')
emotion = pd.read_csv(f_emotion)
emotion.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [4]:
f_sampleSubmission = os.path.join(dir_data, 'sampleSubmission.csv')
sampleSubmission = pd.read_csv(f_sampleSubmission)
sampleSubmission.head()

,id,emotion
0,0x2c7743,surprise
1,0x2c1eed,surprise
2,0x2826ea,surprise
3,0x356d9a,surprise
4,0x20fd95,surprise


### 1.1 split train/ test data

In [55]:
train = data_identification[data_identification['identification'] == 'train'] 
test = data_identification[data_identification['identification'] == 'test'] 

In [56]:
train.head()

,tweet_id,identification
1,0x29e452,train
2,0x2b3819,train
4,0x2a2acc,train
5,0x2a8830,train
6,0x20b21d,train


In [57]:
test.head()

,tweet_id,identification
0,0x28cc61,test
3,0x2db41f,test
15,0x2466f6,test
23,0x23f9e9,test
31,0x1fb4e1,test


In [8]:
f_tweets_DM = os.path.join(dir_data, 'tweets_DM.json')

In [9]:
f = open(f_tweets_DM)
print(f_tweets_DM) 
f = open (f_tweets_DM, "r")
tmp = []
for line in f.readlines():
    dic = json.loads(line)
    tmp.append(dic)

Twitter_kaggle_data/tweets_DM.json


In [11]:
tmp[0]['_source']['tweet']['text']

'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'

In [12]:
tmp_df = pd.DataFrame(tmp)

In [39]:
len(tmp_df)

1867535

## 對_source 分解

In [33]:
from tqdm import tqdm

In [34]:
tqdm.pandas()

In [37]:
tweet = tmp_df['_source'].progress_apply(pd.Series)

100%|███████████████████████████████| 1867535/1867535 [05:16<00:00, 5903.33it/s]


In [38]:
tweet

,tweet
0,"{'hashtags': ['Snapchat'], 'tweet_id': '0x376b..."
1,"{'hashtags': ['freepress', 'TrumpLegacy', 'CNN..."
2,"{'hashtags': ['bibleverse'], 'tweet_id': '0x28..."
3,"{'hashtags': [], 'tweet_id': '0x1cd5b0', 'text..."
4,"{'hashtags': [], 'tweet_id': '0x2de201', 'text..."
...,...
1867530,"{'hashtags': ['mixedfeeling', 'butimTHATperson..."
1867531,"{'hashtags': [], 'tweet_id': '0x29d0cb', 'text..."
1867532,"{'hashtags': [], 'tweet_id': '0x2a6a4f', 'text..."
1867533,"{'hashtags': [], 'tweet_id': '0x24faed', 'text..."


In [42]:
hashtags_tweet_id_text = tweet['tweet'].progress_apply(pd.Series)

100%|███████████████████████████████| 1867535/1867535 [05:17<00:00, 5876.44it/s]


In [43]:
hashtags_tweet_id_text

,hashtags,tweet_id,text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [47]:
dir_data = 'Twitter_kaggle_data/data'
f_data = os.path.join(dir_data, 'hashtags_tweet_id_text.pkl')
hashtags_tweet_id_text.to_pickle(f_data) 

In [48]:
pkl_test = pd.read_pickle(f_data)

In [49]:
pkl_test

,hashtags,tweet_id,text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...,...
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...
1867532,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,[],0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [50]:
tmp_df

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets
...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets


In [46]:
## 備用data
hashtags_tweet_id_text1 = hashtags_tweet_id_text

In [52]:
parsed_twitter = pd.merge(tmp_df, hashtags_tweet_id_text, left_index=True, right_index=True)

In [54]:
parsed_twitter

,_score,_index,_source,_crawldate,_type,hashtags,tweet_id,text
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...,...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets,[],0x29d0cb,I swear all this hard work gone pay off one da...
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets,[],0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [58]:
print(len(train))
train_df = pd.merge(train, parsed_twitter, on='tweet_id',how='left')

1455563


In [63]:
train_df.drop(['identification', '_index', '_type', '_source'], axis=1, inplace=True)

In [64]:
train_df = pd.merge(train_df, emotion, on='tweet_id',how='left')

,tweet_id,_score,_crawldate,hashtags,text,emotion_x,emotion_y
0,0x29e452,809,2015-01-17 03:07:03,[],Huge Respect🖒 @JohnnyVegasReal talking about l...,joy,joy
1,0x2b3819,808,2016-07-02 09:34:06,"[spateradio, app]",Yoooo we hit all our monthly goals with the ne...,joy,joy
2,0x2a2acc,16,2016-08-15 18:18:39,[],@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...,trust,trust
3,0x2a8830,768,2017-02-11 08:49:46,"[PUBG, GamersUnite, twitch, BeHealthy, StayPos...",Come join @ambushman27 on #PUBG while he striv...,joy,joy
4,0x20b21d,70,2016-11-23 05:37:10,"[strength, bones, God]",@fanshixieen2014 Blessings!My #strength little...,anticipation,anticipation
...,...,...,...,...,...,...,...
1455558,0x227e25,361,2016-09-09 14:28:19,[rip],@BBCBreaking Such an inspirational talented pe...,disgust,disgust
1455559,0x293813,15,2017-02-04 14:15:32,"[libtards, Hillary, lost, sad, growup, Trump]",And still #libtards won't get off the guy's ba...,sadness,sadness
1455560,0x1e1a7e,174,2015-12-03 16:53:39,"[seeds, Joy, GLTChurch]",When you sow #seeds of service or hospitality ...,joy,joy
1455561,0x2156a5,515,2016-10-27 03:23:51,[],@lorettalrose Will you be displaying some <LH>...,trust,trust


In [69]:
train_df

,tweet_id,_score,_crawldate,hashtags,text,emotion
0,0x29e452,809,2015-01-17 03:07:03,[],Huge Respect🖒 @JohnnyVegasReal talking about l...,joy
1,0x2b3819,808,2016-07-02 09:34:06,"[spateradio, app]",Yoooo we hit all our monthly goals with the ne...,joy
2,0x2a2acc,16,2016-08-15 18:18:39,[],@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...,trust
3,0x2a8830,768,2017-02-11 08:49:46,"[PUBG, GamersUnite, twitch, BeHealthy, StayPos...",Come join @ambushman27 on #PUBG while he striv...,joy
4,0x20b21d,70,2016-11-23 05:37:10,"[strength, bones, God]",@fanshixieen2014 Blessings!My #strength little...,anticipation
...,...,...,...,...,...,...
1455558,0x227e25,361,2016-09-09 14:28:19,[rip],@BBCBreaking Such an inspirational talented pe...,disgust
1455559,0x293813,15,2017-02-04 14:15:32,"[libtards, Hillary, lost, sad, growup, Trump]",And still #libtards won't get off the guy's ba...,sadness
1455560,0x1e1a7e,174,2015-12-03 16:53:39,"[seeds, Joy, GLTChurch]",When you sow #seeds of service or hospitality ...,joy
1455561,0x2156a5,515,2016-10-27 03:23:51,[],@lorettalrose Will you be displaying some <LH>...,trust


In [70]:
print(len(test))
test_df = pd.merge(test, parsed_twitter, on='tweet_id',how='left')

411972


In [71]:
test_df.drop(['identification', '_index', '_type', '_source'], axis=1, inplace=True)

In [72]:
test_df = pd.merge(test_df, emotion, on='tweet_id',how='left')

In [73]:
test_df

,tweet_id,_score,_crawldate,hashtags,text,emotion
0,0x28cc61,107,2017-01-17 14:13:32,[],@Habbo I've seen two separate colours of the e...,NaN
1,0x2db41f,728,2015-10-17 06:46:20,[],@FoxNews @KellyannePolls No serious self respe...,NaN
2,0x2466f6,491,2016-12-19 03:50:27,[womendrivers],"Looking for a new car, and it says 1 lady owne...",NaN
3,0x23f9e9,28,2017-04-09 19:32:19,[robbingmembers],@cineworld “only the brave” just out and fount...,NaN
4,0x1fb4e1,925,2016-01-15 11:59:31,[],Felt like total dog 💩 going into open gym and ...,NaN
...,...,...,...,...,...,...
411967,0x2c4dc2,792,2017-04-01 02:52:58,[kids],6 year old walks in astounded. Mum! Look how b...,NaN
411968,0x31be7c,34,2016-11-13 07:34:17,[inspiringvolunteerawards2017],Only one week to go until the #inspiringvolunt...,NaN
411969,0x1ca58e,976,2017-10-26 19:12:59,[],"I just got caught up with the manga for ""My He...",NaN
411970,0x35c8ba,534,2017-08-18 10:23:59,[],Speak only when spoken to and make hot ass mus...,NaN


In [75]:
dir_data = 'Twitter_kaggle_data/data'
# store train_df
f_data = os.path.join(dir_data, 'train_df.pkl')
train_df.to_pickle(f_data)
# store test_df
f_data = os.path.join(dir_data, 'test_df.pkl')
test_df.to_pickle(f_data) 

## text preprocess

In [88]:
tweet_tokenizer = TweetTokenizer(reduce_len=True)

In [77]:
train_df['emotion'].value_counts() / len(train_df)

joy             0.354514
anticipation    0.171023
trust           0.141167
sadness         0.132895
disgust         0.095565
fear            0.043969
surprise        0.033478
anger           0.027389
Name: emotion, dtype: float64

In [81]:
s = train_df['text'][0]

In [84]:
s = 'Huge Respect🖒 @JohnnyVegasReal talking about    losing his dad to cancer."if you dont remember the good times it was all for nothing"#beautiful'

In [85]:
s

'Huge Respect🖒 @JohnnyVegasReal talking about        losing his dad to cancer."if you dont remember the good times it was all for nothing"#beautiful'

In [104]:
def text_clean(df):
    #df['text'] = df['text'].apply(lambda s : s.replace('',''))
    
    df['tmp'] = df['text'].apply(lambda s : tweet_tokenizer.tokenize(s))
    # drop @name
    p = re.compile('@*')
    df['tmp'] = df['tmp'].apply(lambda u : ["" if p.match(w).span() != (0, 0) else w for w in u])
    
    # drop hashtag
    p = re.compile('#*')
    df['tmp'] = df['tmp'].apply(lambda u : [" " if p.match(w).span() != (0, 0) else w for w in u])
   
    # replace common emojis with adjectives
    #df['tmp'] = df['tmp'].apply(lambda u : [emoji_dict[w] if w in emojis else w for w in u])

    df['p_text'] = df['tmp'].apply(lambda a : ' '.join(a))
    df.drop(['tmp'], axis=1, inplace=True)
    
    #df['p_text'] = df['p_text'].apply(clean_text)
    df['p_text'] = df['p_text'].str.replace('\d+', 'number',  regex=True)

In [105]:
text_clean(train_df)

/tmp/ipykernel_2473761/1519061083.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df['p_text'] = df['p_text'].str.replace('\d+', 'number')


In [108]:
train_df

,tweet_id,_score,_crawldate,hashtags,text,emotion,p_text
0,0x29e452,809,2015-01-17 03:07:03,[],Huge Respect🖒 @JohnnyVegasReal talking about l...,joy,Huge Respect 🖒 talking about losing his dad t...
1,0x2b3819,808,2016-07-02 09:34:06,"[spateradio, app]",Yoooo we hit all our monthly goals with the ne...,joy,Yooo we hit all our monthly goals with the new...
2,0x2a2acc,16,2016-08-15 18:18:39,[],@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...,trust,Well done team 🌟 <LH> of every one of you .
3,0x2a8830,768,2017-02-11 08:49:46,"[PUBG, GamersUnite, twitch, BeHealthy, StayPos...",Come join @ambushman27 on #PUBG while he striv...,joy,Come join on while he strives for chicken d...
4,0x20b21d,70,2016-11-23 05:37:10,"[strength, bones, God]",@fanshixieen2014 Blessings!My #strength little...,anticipation,Blessings ! My little . My brittle.Door i...
...,...,...,...,...,...,...,...
1455558,0x227e25,361,2016-09-09 14:28:19,[rip],@BBCBreaking Such an inspirational talented pe...,disgust,"Such an inspirational talented person , will ..."
1455559,0x293813,15,2017-02-04 14:15:32,"[libtards, Hillary, lost, sad, growup, Trump]",And still #libtards won't get off the guy's ba...,sadness,"And still won't get off the guy's back , bec..."
1455560,0x1e1a7e,174,2015-12-03 16:53:39,"[seeds, Joy, GLTChurch]",When you sow #seeds of service or hospitality ...,joy,When you sow of service or hospitality in ...
1455561,0x2156a5,515,2016-10-27 03:23:51,[],@lorettalrose Will you be displaying some <LH>...,trust,Will you be displaying some <LH> wares


In [ ]:
df_preprocess(sample_df)
sample_df.head()
df_preprocess(test_df)
df_preprocess(public_test_df)